### DFRでの飛行シミュレーションに相当するトイモデル

#### ルール
- 前の空いてるマスと同じだけ進む（MAX=2）.
- パラレルアップデート
- 参照可能な場合は、前の車の速度を参照しそれを元に自分の速度を決定して良い.
  

In [238]:
import random
import numpy as np

In [243]:
class Car:
    def __init__(self, idx, position):
        self.id = idx
        self.position = position
        self.speed_log = [0]
        self.position_log = [position]
        self.is_active = True
        self.speed = 0

    def decide_speed(self, leader, noise_ratio, policy="simple"):
        """
        前に人がいない場合と十分に車間距離が空いている場合は速度2
        """
        if leader == None:
            if random.random() < noise_ratio:
                self.speed = 0
                return
            self.speed = 2
            return 
        if (leader.position - self.position) >= 3:
            self.speed = 2
            return

        """
        それ以外の場合はPolicyに応じて前の車の速度を用いた速度決定. 
        """
        # DFRの場合
        if policy == "dfr":
            if(leader.position - self.position) >= 2:
                self.speed = leader.speed
                return
            self.speed = leader.speed
            return
        
        # ここは普通のASEPの進み方
        if (leader.position - self.position) >= 2:
            self.speed = 1
            return 
        else:
            self.speed = 0
            return 
        return

    def move(self, total_cell):
        self.position += self.speed
        self.position_log.append(self.position)
        self.speed_log.append(self.speed)

        if self.position > total_cell:
            self.is_active = False
            return self.id
        return -1

class DFRSimulation:

    def __init__(self, **kwargs):
        self.update_rule = kwargs.get("update_rule", "parallel")
        self.initial_position = kwargs.get("INITIAL_POSITION",[])
        self.cars = [Car(idx=idx, position=position) for idx, position in enumerate(self.initial_position)]
        self.TOTAL_CELL = kwargs.get("TOTAL_CELL",40)
        self.TIME_STEPS = kwargs.get("TIME_STEPS",1000)
        self.GLOBAL_DENSITY = kwargs.get("GLOBAL_DENSITY",None)
        self.NOISE_RATIO = kwargs.get("NOISE_RATIO",0.1)
        self.POLICY = kwargs.get("POLICY","simple")

        self.MEAN_SPEED_LOG = []
        self.leading_car_idx = 0

    def step(self):
        """
        εの影響を考慮しない、ただの初等的なASEP.
        """
    
        if self.update_rule == "parallel":
            for idx, car in enumerate(self.cars):
                if idx == self.leading_car_idx:
                    leader = None
                else:
                    leader = self.cars[int(idx-1)]
                car.decide_speed(leader=leader, noise_ratio=self.NOISE_RATIO, policy=self.POLICY)
            for car in self.cars:
                finished_car = car.move(total_cell=self.TOTAL_CELL)
                if finished_car >= 0:
                    self.leading_car_idx = finished_car + 1

        if self.update_rule == "sequential":
            for car in self.cars:
                car.decide_speed()
                car.move(self.TOTAL_CELL)

    def save_by_step(self):
        active_cars = [car for car in self.cars if car.is_active]
        mean_speed = np.mean([car.speed for car in active_cars])
        self.MEAN_SPEED_LOG.append(mean_speed)  

    def arrive(self):
        if random.random() < self.GLOBAL_DENSITY:
            new_car = Car(idx=len(self.cars), position=0)
            self.cars.append(new_car)
            
    def simulate(self):
        for time in range(self.TIME_STEPS):
            if time%20 == 0:
                print("t=",time)
            if self.cars[-1].position > 1:
                self.arrive()
            self.step()
            self.save_by_step()
            

    


In [244]:
CELL_NUM = 100
NUM_CARS = 10
GLOBAL_DENSITY = 0.5
INITIAL_POSITION = sorted(random.sample(range(21), 10), reverse=True)
NOISE_RATIO = 0.1


In [245]:
args = {"update_rule":"parallel", "INITIAL_POSITION":INITIAL_POSITION, "TIME_STEPS":60, "TOTAL_CELL":CELL_NUM, \
       "GLOBAL_DENSITY":GLOBAL_DENSITY, "NOISE_RATIO":NOISE_RATIO}

# 新しいキーと値を追加
args_dfr = {**args, "POLICY": "dfr"}

simulationObj = DFRSimulation(**args)
simulationObj_dfr = DFRSimulation(**args_dfr)


In [246]:
simulationObj.simulate()
simulationObj_dfr.simulate()


t= 0
t= 20
t= 40
t= 0
t= 20
t= 40


In [247]:
print("====SIMPLE=====")
print(len(simulationObj.cars))
print(simulationObj.MEAN_SPEED_LOG)
print()
print("====DFR=====")
print(len(simulationObj_dfr.cars))
print(simulationObj_dfr.MEAN_SPEED_LOG)

====SIMPLE=====
29
[1.0, 1.3, 1.5, 1.5, 1.5, 1.3, 1.4545454545454546, 1.6363636363636365, 1.6363636363636365, 1.6666666666666667, 1.6666666666666667, 1.6923076923076923, 1.6428571428571428, 1.8571428571428572, 1.8666666666666667, 1.8666666666666667, 2.0, 2.0, 1.8823529411764706, 1.8823529411764706, 1.8888888888888888, 1.8421052631578947, 1.894736842105263, 1.9, 1.8571428571428572, 1.9047619047619047, 1.9047619047619047, 1.9090909090909092, 1.8181818181818181, 1.7727272727272727, 1.826086956521739, 1.826086956521739, 1.826086956521739, 1.826086956521739, 1.8333333333333333, 1.8333333333333333, 1.8333333333333333, 1.8333333333333333, 1.84, 1.76, 1.7692307692307692, 1.7692307692307692, 1.7692307692307692, 1.8076923076923077, 1.8518518518518519, 1.8518518518518519, 1.8888888888888888, 1.8846153846153846, 1.8846153846153846, 1.8076923076923077, 1.84, 1.8333333333333333, 1.8333333333333333, 1.826086956521739, 1.8181818181818181, 1.8181818181818181, 1.8095238095238095, 1.8, 1.8, 1.8]

====DFR

In [220]:
a = [4,2,1]
for i,n in enumerate(a):
    print(i,n)
b = [0] * 5
b[2] = 3
print(b)

0 4
1 2
2 1
[0, 0, 3, 0, 0]


In [161]:
random.random()

0.24606190863756439